In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-azure-openai
%pip install llama-index|
%pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
%pip install ipython-ngql networkx pyvis

## Read API keys
All credential information are saved in 'openrc'

In [1]:
# Read credentials

import subprocess
import os

# Define the command to source the openrc file and print environment variables
source_command = 'bash -c ". openrc && env"'

# Run the command and capture its output
completed_process = subprocess.run(source_command, shell=True, stdout=subprocess.PIPE, text=True)

# Parse the output to extract environment variables
env_output = completed_process.stdout
env_lines = env_output.splitlines()
env_variables = {}

for line in env_lines:
    key, value = line.split('=', 1)
    if any([
        "OPENAI" in key,
        "NEBULA" in key,
        "GRAPH" in key,
    ]):
        env_variables[key] = value

os.environ.update(env_variables)


In [2]:
# For OpenAI

import os

# os.environ["OPENAI_API_KEY"], handled in openrc reading

import logging
import sys
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

Settings.llm = llm
Settings.chunk_size = 512

# Building the Knowledge Graph

In [3]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore


from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [4]:
%pip install nebula3-python

os.environ["NEBULA_USER"] = "root"
os.environ[
    "NEBULA_PASSWORD"
] = "nebula"  # replace with your password, by default it is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph 3.5.0 or newer installed locally

# Assume that the graph has already been created
# Create a NebulaGraph cluster with:
# Option 0: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
# Option 1: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext
# and that the graph space is called "paul_graham_essay"
# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE paul_graham_essay(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE paul_graham_essay;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# CREATE TAG INDEX entity_index ON entity(name(256));

space_name = "harry_potter_example1_2"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

Note: you may need to restart the kernel to use updated packages.


### Create a Graph Space
We should make a graph space using nebula

💡 Do not yet have a NebulaGraph cluster? check out nebula-up, it's a one-liner built by me:

`curl -fsSL nebula-up.siwei.io/install.sh | bash`

### Connect to docker-composed nebula

In [5]:
%load_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,harry_potter_example
1,harry_potter_example1_2


#### Create our graph

If you already create a graph with nebula, skip this step.

In [6]:
%ngql CREATE SPACE IF NOT EXISTS harry_potter_example1_2(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [7]:
%%ngql
USE harry_potter_example1_2;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);
CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [8]:
%ngql SHOW SPACES;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,harry_potter_example
1,harry_potter_example1_2


## Instantiate GPTNebulaGraph KG Indexes

In [9]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


If you have previously created a graph in 'storage_graph' folder, you can skip this step

In [10]:
documents = SimpleDirectoryReader(
    input_files=["./data/chapter/1.txt", "./data/chapter/2.txt"]
).load_data()

# Initialize the index
# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
index.storage_context.persist(persist_dir='./storage_graph/ch1_2')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [11]:
from llama_index.core.indices.loading import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph/ch1_2', graph_store=graph_store)
index = load_index_from_storage(
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

INFO:llama_index.core.indices.loading:Loading all indices.


### Querying the Knowledge Graph

In [12]:
query_engine = index.as_query_engine()

response = query_engine.query("Tell me about Tabby Cat")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 54579fb0-4337-41b5-9748-d35bb5ffd38b: Mr. Dursley might have been drifting into an uneasy sleep, but the cat
on the...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>Tabby Cat was sitting as still as a statue on the wall outside, its eyes fixed unblinkingly on the far corner of Privet Drive. It didn't show any sign of sleepiness and remained unmoved even when a man named Albus Dumbledore suddenly appeared on the corner it had been watching. The cat's tail twitched and its eyes narrowed upon seeing Albus Dumbledore, who seemed to find amusement in the sight of the cat.</b>

## Visualizing the Graph RAG

In [14]:
%load_ext ngql

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql


In [15]:
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,harry_potter_example
1,harry_potter_example1_2


In [22]:
%%ngql
USE harry_potter_example1_2;
MATCH p=(n)-[*1..2]-()
  WHERE id(n) IN ['Dursley', 'Owls', 'Berkeley'] 
RETURN p LIMIT 100;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p
0,"(""Owls"" :entity{name: ""Owls""})-[:relationship@..."
1,"(""Dursley"" :entity{name: ""Dursley""})-[:relatio..."
2,"(""Dursley"" :entity{name: ""Dursley""})-[:relatio..."
3,"(""Dursley"" :entity{name: ""Dursley""})-[:relatio..."
4,"(""Dursley"" :entity{name: ""Dursley""})-[:relatio..."
5,"(""Dursley"" :entity{name: ""Dursley""})-[:relatio..."
6,"(""Berkeley"" :entity{name: ""Berkeley""})<-[:rela..."


In [19]:
%ng_draw

<class 'pyvis.network.Network'> |N|=10 |E|=7

In [21]:
%ng_draw

<class 'pyvis.network.Network'> |N|=53 |E|=171